In [1]:
import os
import sys
import json
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import motornet as mn
import utils

from pathlib import Path
from tqdm import tqdm
import argparse
import json    
import task 
import buffer 
from importlib import reload
import agents
reload(agents)
reload(utils)
reload(task)
reload(buffer)
from utils import load_env, load_policy, calc_loss, run_rollout
from task import CentreOutFFGym, CentreOutFF
from buffer import RolloutBuffer
from agents import SLAgent, GRUPPOAgent
import time


In [4]:
TOTAL_TIMESTEPS = 50000
N_STEPS = 2048  # 데이터 수집 스텝 수
HIDDEN_DIM = 128
config = 'params.json'
with open(config, 'r') as f:
    config = json.load(f)

# --- parameters setting ---
env_params = config['env_params']
train_params = config['training_params']

effector = mn.effector.RigidTendonArm26(muscle=mn.muscle.RigidTendonHillMuscle())

# --- 환경 및 에이전트 생성 ---
env = CentreOutFFGym(effector=effector, **env_params)
env_org = CentreOutFF(effector=effector, **env_params)

device = "cuda" if th.cuda.is_available() else "cpu"
agent = GRUPPOAgent(env, hidden_dim=HIDDEN_DIM, device=device)
buffer = RolloutBuffer(env.observation_space.shape[0], env.action_space.shape[0], HIDDEN_DIM, train_params['batch_size'])


print(f"훈련 시작 (총 {TOTAL_TIMESTEPS} 스텝) on {device}...")
start_time = time.time()

obs, _ = env.reset(options={'batch_size': train_params['batch_size']})
hidden_state = agent.network.init_hidden(train_params['batch_size'])


episode_rewards = []
current_episode_reward = 0

훈련 시작 (총 50000 스텝) on cuda...


In [135]:
obs_tensor = th.from_numpy(obs).float().to(device)
action, value, log_prob, next_hidden_state = agent.select_action(obs_tensor, hidden_state)

next_obs, reward, terminated, truncated, info = env.step(action)


In [136]:
buffer.add(obs, action, reward, terminated, log_prob, value)


In [137]:
buffer

In [84]:
states = env.states
goal_th = env.goal

cost_pos = th.sum(th.square(states['fingertip'][:, :2] - goal_th),dim=1)
current_vel = states['cartesian'][:, 2:]
jerk = current_vel - 2 * env.last_vel + env.prev_last_vel
cost_jerk = th.sum(th.square(jerk),dim=1)
muscle_force = states['muscle'][:, 4:5, :]
cost_muscle = th.sum(th.square(muscle_force),dim=2).squeeze()
muscle_force_derivative = muscle_force - env.last_force
cost_muscle_derivative = th.sum(th.square(muscle_force_derivative),dim=2).squeeze()
total_cost = (env.loss_weights['position'] * cost_pos +
                env.loss_weights['jerk'] * cost_jerk +
                env.loss_weights['muscle'] * cost_muscle +
                env.loss_weights['muscle_derivative'] * cost_muscle_derivative)
reward = -(total_cost / 10000)


In [86]:
reward.shape

torch.Size([64])

In [42]:
agent = SLAgent(obs_dim=17, action_dim=6, 
                batch_size = train_params['batch_size'], 
                device='cpu', lr=1e-4)
batch_size = 64
h = agent.policy_net.init_hidden(batch_size = batch_size)
obs, info = env_org.reset(condition='train', options={'batch_size': batch_size})
h.shape, obs.shape
action, h = agent.policy_net(obs,h)


In [45]:
obs, terminated, info = env_org.step(action=action.to('cpu'))

In [31]:
action_tensor.shape

torch.Size([64, 6])

In [17]:
a, b, c = env_org.step(th.from_numpy(action))

IndexError: The shape of the mask [1] at index 0 does not match the shape of the indexed tensor [64, 2] at index 0